In [1]:
class VignereCipher:
    plaintextSpace = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
    
    def __init__(self, secretKey):
        self.key = secretKey
        self.length = len(secretKey)
        
    def __repr(self):
        print "Secret Key:", self.key
    
    def shiftLetters(self, shift, letter, right=True):
        shift = ord(shift) - ord('A')
        shift = shift if right else -shift
        shifted = self.plaintextSpace[(self.plaintextSpace.index(letter) + shift) % 26]
        return shifted

    def encryptMessage(self, message):
        return ''.join([self.shiftLetters(self.key[key_idx % self.length], letter)
                        for key_idx, letter in enumerate(message.upper())])
    
    def decryptMessage(self, message):
        return ''.join([self.shiftLetters(self.key[key_idx % self.length], letter, False)
                    for key_idx, letter in enumerate(message.upper())])   

        

In [2]:
test1 = VignereCipher("CRYPTO")
test1.decryptMessage(test1.encryptMessage("thisisamessage"))

'THISISAMESSAGE'

In [3]:
LETTER_FREQ = [
    ['a',8.167], ['b',1.492], ['c',2.782], ['d',4.253], ['e',12.70], ['f',2.228], ['g',2.015],
    ['h',6.094], ['i',6.966], ['j',0.153], ['k',0.772], ['l',4.025], ['m',2.406], ['n',6.749],
    ['o',7.507], ['p',1.929], ['q',0.095], ['r',5.987], ['s',6.327], ['t',9.056], ['u',2.758],
    ['v',0.978], ['w',2.360], ['x',0.150], ['y',1.974], ['z',0.074]
]
LETTER_FREQUENCY = sorted(map(lambda x: (x[0].upper(), round(x[1], 4)), LETTER_FREQ))

In [233]:
from collections import Counter

class CrackVignere:
    letterFrequency = LETTER_FREQUENCY
    capsletters = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"

    def __init__(self, message):
        self.ciphertext = message

    def lengthProbability(self, length):
        """
        Note: Requires ciphertext be long enough to have a decent sample of letters
        1. Divide ciphertext letters into l groups G0,G1,...,Gl-1
            where ith ciphertext letter is placed in group Gi mod l
        2. Calculate probability of letters in each group being same 
            as a typical English distribution
        """
        # Possibly use Kolmogorov-Smirnov Test, though message size may be too small

        grouped = self.groupList(self.ciphertext, length)
        freqs = [self.distributionError(self.letterFrequency, self.relativeFrequency(group))[1]
                 for group in grouped]
        return sum(freqs) / len(freqs)
        
        
    def groupList(self, lst, size):
        return [lst[i::size] for i in range(size)]
        
    def relativeFrequency(self, sample_dist):
        frequency = Counter(sample_dist)
        total_count = sum(frequency.itervalues())
        rel_frequency = [(k, 100.0 * float(v) / total_count) for k,v in frequency.iteritems()]
        return rel_frequency
    
    def distributionError(self, alst1, alst2):
        a1 = sorted(alst1, key = lambda x: x[1])
        a2 = sorted(alst2, key = lambda x: x[1])
        dist = sum( (a1[i][1] - a2[i][1])**2 for i in range(len(a2)) )
#         print len(a1), len(a2)
        return [(a1[i][0], a2[i][0]) for i in range(len(a2))], dist 
        
    def guessKeyLength(self):
        return sorted([(round(self.lengthProbability(length), 3), length)
                           for length in range(2,20)])
        
    def guessSecretKey(self):
        top3lengths = self.guessKeyLength()[:3]
        print "Top 3 lengths:", top3lengths
        
        toplength = top3lengths[0][1]
        grouped = self.groupList(self.ciphertext, toplength)
        groupedFreq = [self.relativeFrequency(group) for group in grouped]
        
        secretkey = ''
        for idx, group in enumerate(groupedFreq):
            mapper, error = self.distributionError(self.letterFrequency, group)
            diff = int(round(self.meanLetterDiff(mapper)))
            print mapper, idx, diff                                  
            secretkey += self.capsletters[int(round(self.meanLetterDiff(mapper)))]
        
        return secretkey
        
    def meanLetterDiff(self, mapper):
        return mean([ord(pair[0]) - ord(pair[1]) for pair in mapper])  
        
        
        
        

In [234]:
def mean(lst):
    return sum(lst) / float(len(lst))

In [235]:
capsletters = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
f = open('letters.txt', 'r')
r = f.readlines()
sample_msg = filter(lambda x: x in capsletters, ''.join(r).upper())
vignere = VignereCipher("cipher")
cipher = vignere.encryptMessage(sample_msg)
f.close() 
# cipher
# vignere.decryptMessage(cipher)

In [242]:
sample_msg[:10], cipher[:10], vignere.decryptMessage(cipher[:10])

('LETTERSBYT', 'TSOGOOAPTG', 'LETTERSBYT')

In [236]:
cracker = CrackVignere(cipher)

In [237]:
cracker.guessSecretKey()

Top 3 lengths: [(1.97, 6), (2.003, 12), (2.023, 18)]
[('Z', 'H'), ('Q', 'Y'), ('X', 'R'), ('J', 'F'), ('K', 'S'), ('V', 'D'), ('B', 'J'), ('P', 'O'), ('Y', 'E'), ('G', 'G'), ('F', 'X'), ('W', 'U'), ('M', 'N'), ('U', 'C'), ('C', 'K'), ('L', 'L'), ('D', 'T'), ('R', 'P'), ('H', 'Z'), ('S', 'A'), ('N', 'V'), ('I', 'I'), ('O', 'Q'), ('A', 'W'), ('T', 'B'), ('E', 'M')] 0 0
[('Z', 'N'), ('Q', 'E'), ('X', 'X'), ('J', 'L'), ('K', 'Y'), ('V', 'J'), ('B', 'P'), ('P', 'U'), ('Y', 'K'), ('G', 'M'), ('F', 'D'), ('W', 'A'), ('M', 'T'), ('U', 'I'), ('C', 'Q'), ('L', 'R'), ('D', 'Z'), ('R', 'V'), ('H', 'F'), ('S', 'G'), ('N', 'B'), ('I', 'O'), ('O', 'W'), ('A', 'C'), ('T', 'H'), ('E', 'S')] 1 0
[('Z', 'U'), ('Q', 'L'), ('X', 'E'), ('J', 'S'), ('K', 'F'), ('V', 'Q'), ('B', 'B'), ('P', 'W'), ('Y', 'R'), ('G', 'T'), ('F', 'K'), ('W', 'H'), ('M', 'A'), ('U', 'P'), ('C', 'X'), ('L', 'Y'), ('D', 'G'), ('R', 'C'), ('H', 'M'), ('S', 'N'), ('N', 'I'), ('I', 'V'), ('O', 'D'), ('A', 'J'), ('T', 'O'), ('E', 'Z')] 

'AAAAAA'

In [224]:
x = [('Z', 'R'), ('Q', 'Y'), ('X', 'F'), ('J', 'D'), ('K', 'S'), ('V', 'J'), ('B', 'U'), ('P', 'X'), ('Y', 'N'), ('G', 'E'), ('F', 'K'), ('W', 'O'), ('M', 'G'), ('U', 'C'), ('C', 'L'), ('L', 'P'), ('D', 'T'), ('R', 'Z'), ('H', 'V'), ('S', 'A'), ('N', 'Q'), ('I', 'I'), ('O', 'W'), ('A', 'B'), ('T', 'M')]

In [227]:
def mld(mapper):
    return mean([ord(pair[0]) - ord(pair[1]) for pair in mapper]) 

In [228]:
mld(x)

0.12

In [229]:
ord('Z') - ord('R')

8